importazione necessaria per visualizzare decision trees e DNN map

In [1]:
!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [2]:
import ROOT
from ROOT import TMVA, TCut, TFile
from math import floor

Welcome to JupyROOT 6.22/03


necessario per usare interactive features in notebook 

In [3]:
%jsmva on

dichiarazione dell Factory analoga alla versione utilizzata in TMVA in C++. abilitando JsMVA abbiamo la possibilità di utilizzare una versione Python del costruttore definito in (VEDI GUIDA) 

In [4]:
outputFile = TFile( "TMVAoutput.root", 'RECREATE' )
factory = TMVA.Factory("TMVAClassification", TargetFile=outputFile,
                       V=False, Color=True, DrawProgressBar=True, Transformations=["I", "D", "P", "G", "D"],
                       AnalysisType="Classification")

Ora dichiariamo un DataLoader e aggiungiamo le variabili usando la AddVariable Function

In [5]:
dataset = "dataset" #the dataset name
dataloader  = TMVA.DataLoader(dataset)

dataloader.AddVariable( "massK0S", 'F' )
dataloader.AddVariable( "tImpParBach", 'F' )
dataloader.AddVariable( "tImpParV0", 'F' )
dataloader.AddVariable( "CtK0S := DecayLengthK0S*0.497/v0P", 'F')
dataloader.AddVariable( "cosPAK0S", 'F' )
dataloader.AddVariable( "CosThetaStar", 'F' )
dataloader.AddVariable( "nSigmaTOFpr", 'F' )
dataloader.AddVariable( "nSigmaTOFpi", 'F' )
dataloader.AddVariable( "nSigmaTOFka", 'F' )
dataloader.AddVariable( "nSigmaTPCpr", 'F' )
dataloader.AddVariable( "nSigmaTPCpi", 'F' )
dataloader.AddVariable( "nSigmaTPCka", 'F' )

#spectator variables

dataloader.AddSpectator( "massLc2K0Sp", "mass Lc-->K0S+p", 'F' )
dataloader.AddSpectator( "LcPt",  "Lc Pt", 'F' )
dataloader.AddSpectator( "massLc2Lambdapi",  "mass Lc -->L(1520)+pi", 'F' )
dataloader.AddSpectator( "massLambda", "mass V0 = Lambda", 'F' )
dataloader.AddSpectator( "massLambdaBar", "mass V0 = LambdaBar", 'F' )
dataloader.AddSpectator( "V0positivePt", "V0 positive Pt", 'F' )
dataloader.AddSpectator( "V0negativePt", "V0 negative Pt", 'F' )
dataloader.AddSpectator( "dcaV0pos", "dca V0 positive", 'F' )
dataloader.AddSpectator( "dcaV0neg", "dca V0 negative", 'F' )
dataloader.AddSpectator( "v0Pt", "K0S Pt", 'F' )
dataloader.AddSpectator( "dcaV0", "dca V0", 'F' )
dataloader.AddSpectator( "V0positiveEta", "V0pos Eta", 'F' )
dataloader.AddSpectator( "bachelorEta", "eta bachelor", 'F' )

apriamo il dataset contenente signal and background trees

In [6]:
input = TFile.Open( "AnalysisResults.root")

In [7]:
# signal and backg trees for training
signal      = input.Get( "treeList_2_25_2_25_Sgn")
background  = input.Get( "treeList_2_25_2_25_Bkg")

global event weights per tree (see below for setting event-wise weights)
Diamo al Training e al Test lo stesso tree, questo verrà quindi diviso in due sottounità che serviranno separatamente per il training e il test. 

In [17]:
signalWeight     = 1.0
backgroundWeight = 1.0

dataloader.AddSignalTree(signal, signalWeight)
dataloader.AddBackgroundTree(background, backgroundWeight)

#dataloader.fSignalWeight = signalWeight
#dataloader.fBackgroundWeight = backgroundWeight
#dataloader.fTreeS = signal
#dataloader.fTreeB = background

Add Tree treeList_2_25_2_25_Sgn of type Signal with 1762028 events
Add Tree treeList_2_25_2_25_Bkg of type Background with 1473789 events


aggiungo dei Tagli sull'impulso trasverso per selezionare solo una parte degli eventi presenti nel tree

In [18]:
ptmin=8
ptmax=12
mycuts = TCut("LcPt<ptmax && LcPt>ptmin")
mycutb = TCut("LcPt<ptmax && LcPt>ptmin")

Applicato il taglio sul momento trasverso il numero di eventi da utilizzare nel training e nel test viene deciso come segue:

1) il training e il testing del segnale viene fatto con il valore minimo tra la metà degli eventi rimasti dopo il taglio e 100000

2) il training e il testing del Bakground viene fatto con il valore minimo tra la metà degli eventi rimasti dopo il taglio e 200000

Training e testing vengono ovviamente fatti con eventi diversi selezionati in maniera randomica all'interno del tree

In [19]:
nTrainingEventsSgn = floor(min(signal.GetEntries("LcPt<12 && LcPt>8")*0.5,100000))
nTrainingEventsBkg = floor(min(background.GetEntries("LcPt<12 && LcPt>8")*0.5,200000))
nTestingEventsSgn = floor(min(signal.GetEntries("LcPt<12 && LcPt>8")*0.5,100000))
nTestingEventsBkg = floor(min(background.GetEntries("LcPt<12 && LcPt>8")*0.5,200000))
print(nTrainingEventsSgn)

71245


usiamo la funzione DataLoader.PrepareTrainingAndTestTree  per applicare i cut agli eventi di input

In [20]:
dataloader.PrepareTrainingAndTestTree(SigCut=mycuts, BkgCut=mycutb,nTrain_Signal=nTrainingEventsSgn, 
                        nTest_Signal=nTestingEventsSgn, nTrain_Background=nTrainingEventsBkg , 
                        nTest_Background=nTestingEventsBkg, SplitMode="Random", NormMode="NumEvents", V=False)